# R.BrowserUnit Experimentation Notebook

**R.BrowserUnit** is a simple python library class for running Selenium Webdriver tests inside UnitTest functions. It is intended to speed the automation of simple and repetitive browser actions.

This notebook is a collection of experimental methods to develop a minimal set of functions for quickly checking for element content, element presence, switching pages, parsing sitemaps, etc.


In [1]:
import rcf_lib as R
import unittest
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options as C_Options
from selenium.webdriver.firefox.options import Options as FF_Options
import time
import sys
from colorama import init as cinit
cinit() # Filters ANSI escape sequences and inserts Win32 calls.

In [2]:
x = "Hello, world!"

In [3]:
print(x)

Hello, world!


In [4]:
FF_BINARY_PATH = "webdrivers/geckodriver.exe" # Assuming notebook was opened from py/
CHROME_BINARY_PATH = "webdrivers/chromedriver.exe"

def FF_DRIVER():
    firefox_options = FF_Options()
    return webdriver.Firefox(executable_path=FF_BINARY_PATH,options=firefox_options)

def C_DRIVER():
    chrome_options = C_Options()
    chrome_options.add_argument("--window-size=200,")
    chrome_options.add_argument("disable-infobars")
    chrome_options.add_argument("--disable-notifications")
    return webdriver.Chrome(executable_path=CHROME_BINARY_PATH,options=chrome_options)

At this point, `FF_DRIVER()` will open a FireFox window for testing, and `CHROME_DRIVER()` will open a Chrome window for testing. Both functions return the **driver** that is used for maniuplating the page. We'll attempt to open and manipulate the FireFox driver first. A window should open after this command is run.

In [6]:
# Test ChromeDriver
c_driver = C_DRIVER()
c_driver.get("http://catb.org/jargon/html/index.html")

In [7]:
# Test GeckoDriver
ff_driver = FF_DRIVER()
ff_driver.get("http://catb.org/jargon/html/index.html")